In [1]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)


C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tqdm
from datasets import load_dataset, Dataset

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('./output.csv')

In [4]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
dataset_train.reset_index(drop=True, inplace=True)
dataset_test.reset_index(drop=True, inplace=True)

In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR = 'results'
MAX_LENGTH = 512

In [9]:
rouge = evaluate.load("rouge")

In [10]:
def summarize_text(text, max_input_length=512, max_output_length=25):
    """
    Summarizes input text using a T5 model.
    """
    input_text = text.strip()
    
    # Tokenize
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to(device)
    
    # Generate the summary
    outputs = model.generate(inputs, max_length=max_output_length, min_length=5, length_penalty=2.0, num_beams=4)
    
    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [11]:
data = df[:500] # use only 500 rows for testing finetuning process

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data, test_size=0.2, random_state=42)

In [12]:
dataset_train.reset_index(drop=True, inplace=True)
dataset_test.reset_index(drop=True, inplace=True)

In [13]:
dataset_test["generated_summary"] = dataset_test["preprocessed_text"].apply(summarize_text)

In [14]:
results = rouge.compute(
    predictions=dataset_test["generated_summary"].tolist(),
    references=dataset_test["Summary"].tolist()
)

In [15]:
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.0927
rouge2: 0.0264
rougeL: 0.0869
rougeLsum: 0.0861


In [16]:
def preprocess_data(row, tokenizer, max_input_length=512, max_target_length=50):
    """
    Tokenize a single row for T5 training.
    """

    inputs = row["preprocessed_text"] 
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    
    labels = tokenizer(row["Summary"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs


In [17]:
tokenized_train = dataset_train.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

tokenized_test = dataset_test.apply(
    lambda row: preprocess_data(row, tokenizer),
    axis=1
)

In [18]:
tokenized_train


0      [input_ids, attention_mask, labels]
1      [input_ids, attention_mask, labels]
2      [input_ids, attention_mask, labels]
3      [input_ids, attention_mask, labels]
4      [input_ids, attention_mask, labels]
                      ...                 
395    [input_ids, attention_mask, labels]
396    [input_ids, attention_mask, labels]
397    [input_ids, attention_mask, labels]
398    [input_ids, attention_mask, labels]
399    [input_ids, attention_mask, labels]
Length: 400, dtype: object

In [19]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="t5_summarization", 
    evaluation_strategy="steps",    
    eval_steps=50,                 
    logging_dir="./logs",           
    logging_steps=10,              
    save_steps=20,                
    save_total_limit=5,             
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    num_train_epochs=30,             
    learning_rate=5e-5,             
    weight_decay=0.01,             
    warmup_steps=50,               
    fp16=torch.cuda.is_available(),  
    report_to="none"                
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)


C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\shiva\AppData\Local\Temp\ipykernel_8944\3505941272.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
history = trainer.train()

  1%|          | 10/1500 [00:10<17:29,  1.42it/s]

{'loss': 17.4645, 'grad_norm': 55.47995376586914, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.2}


  1%|▏         | 20/1500 [00:17<20:03,  1.23it/s]

{'loss': 17.3244, 'grad_norm': 51.48433303833008, 'learning_rate': 1.7000000000000003e-05, 'epoch': 0.4}


  2%|▏         | 30/1500 [00:33<34:40,  1.42s/it]

...

{'loss': 0.3436, 'grad_norm': 1.0837959051132202, 'learning_rate': 1.7241379310344828e-07, 'epoch': 30.0}


                                                   
100%|██████████| 1500/1500 [12:19<00:00,  3.93it/s]

{'eval_loss': 0.5391696691513062, 'eval_runtime': 1.0137, 'eval_samples_per_second': 98.645, 'eval_steps_per_second': 12.824, 'epoch': 30.0}


100%|██████████| 1500/1500 [12:20<00:00,  2.03it/s]

{'train_runtime': 740.6976, 'train_samples_per_second': 16.201, 'train_steps_per_second': 2.025, 'train_loss': 0.949371881643931, 'epoch': 30.0}


In [69]:
def summarize_text_finetuned(text, max_input_length=512, max_output_length=10):
    """
    Summarizes input text using a T5 model.
    """
    # Add the "summarize:" prefix for T5
    input_text = text.strip()
    
    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to(device)
    
    # Generate the summary
    outputs = trainer.model.generate(inputs, max_length=max_output_length, min_length=1, length_penalty=1.0, num_beams=4)
    
    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [36]:
tokenizer.save_pretrained("Finetuned_T5/tokenizer")
trainer.model.save_pretrained("Finetuned_T5/model")

In [208]:
dataset_train["generated_summary"] = dataset_train["preprocessed_text"].apply(summarize_text)

In [70]:
dataset_test["generated_summary"] = dataset_test["preprocessed_text"].apply(summarize_text_finetuned)

In [72]:
dataset_test["generated_summary"]

0                      Not my cup of coffee
1              Maltitol is an alcohol sugar
2             Best chicken noodle soup ever
3               Great for green tomato jam!
4        A nice alternative to an apple pie
                      ...                  
95              Great for the lowest price!
96            My dog has a ton of allergies
97                    No tea flavor at all.
98    My favorite dairy free flavored chips
99                     Strawberry Twizzlers
Name: generated_summary, Length: 100, dtype: object

In [73]:
results = rouge.compute(
    predictions=dataset_test["generated_summary"].tolist(),
    references=dataset_test["Summary"].tolist()
)

In [74]:
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1563
rouge2: 0.0403
rougeL: 0.1469
rougeLsum: 0.1462


There's an increase in ROUGE score.
Since it's t5-small it's to get score more than this